# Content
1. ['Optimizing Portfolio Sharp'](#1)
- Given a list of tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'BRK-A', 'V', 'JNJ', 'WMT']

- Highest sharpe ratio of 1.07 is attained with high weight in MSFT (40%).  The performance of MSFT dominates the portfolio. TSLA, JNJ and WMT have weights of 12%, the rest are less than 10%. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import warnings
warnings.filterwarnings('ignore')

# 1. Optimizing Portfolio Sharpe
<a id='1'></a>

- Given a list of tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'BRK-A', 'V', 'JNJ', 'WMT']

- Highest sharpe ratio of 1.07 is attained with high weight in MSFT (40%).  The performance of MSFT dominates the portfolio. TSLA, JNJ and WMT have weights of 12%, the rest are less than 10%. 


In [ ]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'BRK-A', 'V', 'JNJ', 'WMT']

for ticker in tickers:
    df = pdr.get_data_yahoo(ticker, start='2013-01-01', end='2023-11-18')
    df = df[['Adj Close']]
    df = df.rename(columns={'Adj Close': ticker})
    
    if ticker == tickers[0]:
        dfmerge = df
    else:
        dfmerge = dfmerge.join(df, how='outer')

In [ ]:
dfmerge.head()

In [ ]:
dfret = dfmerge.pct_change().apply(lambda x: np.log(1+x)).dropna()

In [ ]:
fig, ax1 = plt.subplots(ncols=2, nrows=5, figsize=(15, 20))  # Set the figure size here
ax = ax1.flatten()

for col, ax in zip(dfret.columns, ax):
    ax.plot(dfret[col])
    ax.set_title(col)

In [ ]:
import seaborn as sns
sns.heatmap(dfret.corr())

In [ ]:
dfcumuret = dfret.cumsum().apply(np.exp)
dfcumuret.head()

In [ ]:
dfcumuret.plot()

In [ ]:
tickers_exTSLA = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'BRK-A', 'V', 'JNJ', 'WMT']

dfcumuret_exTSLA = dfcumuret[tickers_exTSLA]
dfcumuret_exTSLA.plot()

# Find stock weights (w) that minimize portfolio volatility

In [ ]:
import scipy
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

bounds = Bounds(0,1) # all weights between 0 and 1
linear_constraint = LinearConstraint(np.ones(10), 1,1) # sum of weight >1 and <1, hence sum of weight = 1 (equality constraint)
covar=dfret.cov()
r=np.mean(dfret,axis=0)*252 # r = mean return of each stock

weights = np.ones(10)
x0 = [0.1]*10 # initial guess of respective weights

portfstrderr = lambda w: np.sqrt(np.dot(w, np.dot(w,covar))*252)

res1 = minimize(portfstrderr, x0, method='trust-constr', constraints=linear_constraint, bounds=bounds)

# methods?
- <u/>trust-constr</u>
    - equality and inequality constraints
    - (eg) optimize portfolio weights, given that sum of weights equal 1
- <u/>BFGS</u>
    - unconstrained 
    - (eg) minimum of a quadratic function
- <u/>L-BFGS-B</u>
    - bound constraints

In [ ]:
def ret(r,w):
    return r.dot(w)

def vol(w,covar):
    return np.sqrt(np.dot(w, np.dot(w,covar))*252)

def sharpe(ret, vol):
    return ret/vol

w_min = res1.x
ret1 = ret(r,w_min)
vol1 = vol(w_min,covar)
sharpe1 = sharpe(ret1, vol1)
print('return = ', round(ret1,2))
print('vol = ', round(vol1,2))
print('sharpe ratio = ', round(sharpe1,2))

# Maximize sharpe ratio

In [ ]:
invSharpe = lambda w: np.sqrt(np.dot(w, np.dot(w,covar))*252)   /    r.dot(w)
res2 = minimize(invSharpe, x0, method='trust-constr', constraints=linear_constraint, bounds=bounds)

In [ ]:
w_Sharpe = res2.x
ret2 = ret(r,w_Sharpe)
vol2 = vol(w_Sharpe,covar)
sharpe2 = sharpe(ret2, vol2)
print('return = ', round(ret2,2))
print('vol = ', round(vol2,2))
print('sharpe ratio = ', round(sharpe2,2))

# Analyse relationship of return, volatility and sharpe ratio

In [ ]:
w = w_min
num_ports = 100
r_min = ret(r,w_min)
gap = (max(r) - r_min) / num_ports

all_weights = np.zeros((num_ports, len(dfret.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)

x0= w_min

for i in range(num_ports):
    port_ret = r_min + i*gap 
    double_constraints = LinearConstraint([np.ones(dfret.shape[1]), r], [1,port_ret], [1,port_ret])
    # equality constraint : sum of weight == 1
    # equality constraint : portfolio return == port_ret
    portfvola = lambda w: np.sqrt(np.dot(w,np.dot(w,covar))*252)
    res = minimize(portfvola, x0, method='trust-constr', constraints=double_constraints, bounds=bounds)
    all_weights[i,:] = res.x # populate the weights for diff i
    ret_arr[i] = port_ret
    vol_arr[i] = vol(res.x, covar)
    
sharpe_arr = ret_arr/ vol_arr

plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')

In [ ]:
df_sharpe = pd.DataFrame({'sharpe':sharpe_arr, 
                          'ret_arr':ret_arr, 'vol_arr':vol_arr})

In [ ]:
all_weights_df = pd.DataFrame(all_weights)
all_weights_df.columns = tickers

In [ ]:
df_sharpe = df_sharpe.join(all_weights_df, how='left')

In [ ]:
df_sharpe = df_sharpe.sort_values('sharpe',ascending=False)

In [ ]:
print(tickers)

for i in range(10):
    # 0 - 9
    # 10 - 19
    # 90-99
    ave_sharpe = df_sharpe['sharpe'][i*10:i*10+10].mean()
    print()
    print('average sharpe = ', round(ave_sharpe,2))
    
    ave_weights = []
    for each in tickers:
        ave_weight = df_sharpe[each][i*10:i*10+10].mean()
        ave_weights.append(round(ave_weight,3))
        
    print('ave_weights')
    print(ave_weights)

In [ ]:
weights = [0.092, 0.393, 0.016, 0.0, 0.001, 0.138, 0.119, 0.037, 0.118, 0.086]

In [ ]:
pd.DataFrame({'Tickers':tickers, 'Weights':weights}).sort_values('Weights', ascending=False)

<font color='red'> Highest sharpe ratio of 1.07 is attained with high weight in MSFT (40%).  The performance of MSFT dominates the portfolio. TSLA, JNJ and WMT have weights of 12%, the rest are less than 10%. 

# Thank you for reading :)